In [ ]:
# Automatically import the libraries into Jupyter Notebook
%run ../functions/startup.py
from _funct_annotat import Labels_proc
lab_p = Labels_proc()

In [ ]:
def get_parent_dir(directory):
    import os
    return os.path.dirname(directory)

current_dirs_parent = get_parent_dir(os.getcwd())

In [ ]:
path_base = str(current_dirs_parent)
folder = '/data_examples/UWO_data/'  # Select data folder

path_all = path_base + folder
save_path = path_all     # Destination folder to for labelled data
name_of_file1 = 's1.csv'

name_of_file_l1_time = "Labels_Anomaly_"

### Functions

In [ ]:
# ADD zeros and ones with dictionary mapping

mapper_dict = {'left_only': 0, 'both': 1}
def mp(entry):
    """
    maps new values
    """
    return mapper_dict[entry] if entry in mapper_dict else entry
mp = np.vectorize(mp)


def comp_labs_case1 (data_lab_1, data1_l):
    labels_df_1 = pd.merge(data_lab_1,data1_l, on = ['date', 'time'], how='left', indicator=True)
    labels_df_1 ['_merge'] = mp(labels_df_1['_merge'])
    labels_df_1 = labels_df_1.rename(index=str, columns={"_merge": "Anomaly"})
    
    data_1 = labels_df_1.copy()
    data_1['datetime'] = pd.to_datetime(labels_df_1['date']+ ' ' + labels_df_1['time'])
    data_1.drop_duplicates('datetime',inplace=True)
    if data_lab_1.shape[1] == 5:
        print('Fraction of collective anomalies bf_03, bf_04:', np.sum(data_1['Anomaly'] ==1)/data_1.shape[0])
    else:
        print('Fraction of anomalies in ', data_lab_1.keys()[2], ': ', np.sum(data_1['Anomaly'] ==1)/data_1.shape[0])
    print('Shape:', data_1.shape)
    data_1.reset_index(inplace=True)
    
    return data_1

# Load, explore and manipulate data

In [ ]:
completePath = os.path.join(path_all, name_of_file1) 
df = pd.read_csv(completePath)

In [ ]:
df2 = df.copy(deep=True)
df2['date'], df2['time'] = df2['time'].str.split(' ', 1).str

sr0 = df2.keys()[1]
sr1 = df2.keys()[2]
sr2 = df2.keys()[3]
print('Sensor names:',sr0,',', sr1,',', sr2)

# Create datetime

df2['date'] = [x.date() for x in (pd.to_datetime([i for i in df2['date']], format='%Y-%m-%d'))] 
df2['time'] = [x.time() for x in (pd.to_datetime([i for i in df2['time']], format='%H:%M:%S'))]   # remove primes from the time
df1 = df2.copy(deep=True)
df2.set_index(['date','time'], inplace=True)

df_bf_00 = df2[sr0]
df_bf_01 = df2[sr1]
df_bf_02 = df2[sr2]

In [ ]:
# Remove dates where temporal resolution is wrong
#df2, date_list = lab_p.rem_dates(df2, df_bf_00, drop = False)

# Get date_time
data_df = df2.copy()
data_lab_1 = lab_p.get_date_time(data_df, sr0)
data_lab_2 = lab_p.get_date_time(data_df, sr1)
data_lab_3 = lab_p.get_date_time(data_df, sr2)
data_lab_123 = lab_p.get_date_time(data_df, sr0, multi= 1)

### Load and compute labels

In [ ]:
data1_l = pd.read_csv(os.path.join(path_all, name_of_file_l1_time +sr0 + '.csv'))  
data2_l = pd.read_csv(os.path.join(path_all, name_of_file_l1_time +sr1 + '.csv'))  
data3_l = pd.read_csv(os.path.join(path_all, name_of_file_l1_time +sr2 + '.csv')) 
data123_l = pd.read_csv(os.path.join(path_all, name_of_file_l1_time +sr0 + sr1 +sr2 + '.csv')) 

In [ ]:
data1_l.drop(['Unnamed: 0', data1_l.keys()[3],'Anomaly' ], axis=1, inplace=True)
data2_l.drop(['Unnamed: 0',data2_l.keys()[3],'Anomaly' ], axis=1, inplace=True)
data3_l.drop(['Unnamed: 0',data3_l.keys()[3],'Anomaly' ], axis=1, inplace=True)
data123_l.drop(['Unnamed: 0',data123_l.keys()[3],data123_l.keys()[4],data123_l.keys()[5],'Anomaly' ], axis=1, inplace=True)

In [ ]:
data_1 = comp_labs_case1 (data_lab_1, data1_l)
data_2 = comp_labs_case1 (data_lab_2, data2_l)
data_3 = comp_labs_case1 (data_lab_3, data3_l)
data_123 = comp_labs_case1 (data_lab_123, data123_l)

In [ ]:
### merge anomalies

TT = data_1.Anomaly + data_2.Anomaly + data_123.Anomaly
for i, x in enumerate(TT):
    if x >= 1: TT[i] = 1
        
list_df = list(zip(data_123.datetime, data_123.bf_03, data_123.bf_04, data_123.bl_ce193, TT))
merg_data_df = pd.DataFrame(list_df, columns=['datetime', 'bf_03', 'bf_04', 'bl_ce193','Tot_anomalies'])

## Save data

In [ ]:
# # Save data as pickle

name = 'UWO_Anomaly_collective' 
lab_p.save_pkl(name,  path_base, folder, merg_data_df)

In [ ]:
data_1 = lab_p.rearr_cols(data_1)
data_2 = lab_p.rearr_cols(data_2)
data_3 = lab_p.rearr_cols(data_3)
data_123 = lab_p.rearr_cols(data_123)

In [ ]:
# # Save data as pickle

name1 = 'bf_03_Anomaly' 
name2 = 'bf_04_Anomaly' 
name3 = 'bl_ce193_Anomaly' 
name4 = 'bf_03_bf_04_bl_ce193_Anomaly' 

lab_p.save_pkl(name1,  path_base, folder, data_1)
lab_p.save_pkl(name2, path_base, folder, data_2)
lab_p.save_pkl(name3,  path_base, folder, data_3)
lab_p.save_pkl(name4,  path_base, folder, data_123)